In [ ]:
import json
import re
import os
from pymatgen.ext.matproj import MPRester
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from mat3ra.made.tools.build.interface import InterfaceConfiguration, ZSLStrainMatchingParameters, \
    ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters
from utils.visualize import visualize_materials
from utils.jupyterlite import set_materials

from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material
from mat3ra.made.tools.modify import wrap_to_unit_cell

materials = Materials.get_by_categories("3D")

# Load the symbols from the JSON file
with open('symbols_map.json', 'r') as file:
    symbols = json.load(file)

# Extract the "OVER" symbol
SLASH_SYMBOL = symbols["/"]

# Map lattice type to Miller Index

lattice_to_miller = {
    "CUB": (1, 1, 1),
    "FCC": (1, 1, 1),
    "HEX": (0, 0, 1),
    "TRI": (1, 1, 1)
}


def generate_interface(substrate_json, film_json):
    print(f"Creating interface: {substrate_json['name']} and {film_json['name']}")

    substrate = Material(substrate_json)
    film = Material(film_json)
    substrate_dimensionality = "2D" if "2D" in substrate.name else "3D"
    film_dimensionality = "2D" if "2D" in film.name else "3D"

    substrate_miller_indices = lattice_to_miller[substrate.lattice.type] if substrate.lattice.type in lattice_to_miller else (0, 0, 1)
    film_miller_indices = lattice_to_miller[film.lattice.type] if film.lattice.type in lattice_to_miller else (0, 0, 1)

    # Get material names before the "," -- the formula
    substrate_name = re.match(r'[^,]*', substrate.name).group(0) + str(substrate_miller_indices).replace(", ", "")
    film_name = re.match(r'[^,]*', film.name).group(0) + str(film_miller_indices).replace(", ", "")

    interface_name = f"{film_name}{SLASH_SYMBOL}{substrate_name}"
    print(f"Interface name: {interface_name}")

    # Define slab and interface parameters
    film_params = {
        "miller_indices": film_miller_indices,
        "thickness": 1 if film_dimensionality == "2D" else 3,
        "vacuum": 15.0,
        "xy_supercell_matrix": [[1, 0], [0, 1]],
        "use_conventional_cell": True,
        "use_orthogonal_z": True
    }

    substrate_params = {
        "miller_indices": substrate_miller_indices,
        "thickness": 1 if substrate_dimensionality == "2D" else 3,
        "vacuum": 15.0,
        "xy_supercell_matrix": [[1, 0], [0, 1]],
        "use_conventional_cell": True,
        "use_orthogonal_z": True
    }


    interface_params = {
        "distance_z": 3.0,
        "vacuum": 3.0,
        "max_area": 150,
        "max_area_tol": 0.10,
        "max_angle_tol": 0.04,
        "max_length_tol": 0.04
    }


    # Create slab configurations
    substrate_slab_config = SlabConfiguration(bulk=substrate, **substrate_params)
    film_slab_config = SlabConfiguration(bulk=film, **film_params)
    try:
        # Get terminations
        substrate_terminations = get_terminations(substrate_slab_config)
        film_terminations = get_terminations(film_slab_config)

        # Create slabs
        # substrate_slabs = [create_slab(substrate_slab_config, t) for t in substrate_terminations]
        # film_slabs = [create_slab(film_slab_config, t) for t in film_terminations]

        # Select termination pair (example: first pair)
        termination_pair = (film_terminations[0], substrate_terminations[0])

        # Create interface configuration
        interface_config = InterfaceConfiguration(
            film_configuration=film_slab_config,
            substrate_configuration=substrate_slab_config,
            film_termination=termination_pair[0],
            substrate_termination=termination_pair[1],
            distance_z=interface_params["distance_z"],
            vacuum=interface_params["vacuum"]
        )

        # Set strain matching parameters
        zsl_params = ZSLStrainMatchingParameters(
            max_area=interface_params["max_area"],
            max_area_tol=interface_params["max_area_tol"],
            max_angle_tol=interface_params["max_angle_tol"],
            max_length_tol=interface_params["max_length_tol"]
        )

        # Generate interfaces
        builder = ZSLStrainMatchingInterfaceBuilder(
            build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(strain_matching_parameters=zsl_params)
        )
        interfaces = builder.get_materials(configuration=interface_config)

        # Visualize and save the interfaces
        interface = interfaces[0]
        interface = wrap_to_unit_cell(interface)
        visualize_materials([{"material": interface, "rotation" : "-90x"},{"material": interface}])

        interface.name = interface_name
        set_materials(interface)
    except Exception as e:
        print(f"Error creating interface between {substrate.name} and {film.name}: {e}")


## Generate interfaces between all pairs of materials

In [ ]:
# Create interfaces
# for i, substrate_json in enumerate(materials):
#     for j, film_json in enumerate(materials):
#         if i != j:
#             generate_interface(substrate_json, film_json)


## Generate interfaces between random pairs of materials

In [ ]:
import random
num_pairs = 20

for _ in range(num_pairs):
    substrate_json, film_json = random.sample(materials, 2)
    generate_interface(substrate_json, film_json)